# Network Viewer

NOTE: this network viewer requires installation of two utilities outside of python:

1. [Tippecanoe](https://github.com/mapbox/tippecanoe)
2. [mbview](https://github.com/mapbox/mbview)*


\* Note that mbview is a Node.js application. Please [download and install node.js first](https://nodejs.org/en/download/) if you don't already have it.

After that, you can use the shell commands (details dependent on computer operating system):

```sh
brew install tippecanoe
npm install -g @mapbox/mbview
```

In [3]:
import os
import sys
from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import net_to_mapbox
%config IPCompleter.greedy=True

import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
# if you don't want to see so much detail, set to logging.INFO or DEBUG if you want to see everything
logger.setLevel(logging.DEBUG)

In [4]:
%load_ext autoreload
%autoreload 2

## Set Directories

In [5]:
STPAUL_DIR = os.path.join(os.path.dirname(os.getcwd()),'examples','stpaul')
STPAUL_SHAPE_FILE = os.path.join(STPAUL_DIR,"shape.geojson")
STPAUL_LINK_FILE = os.path.join(STPAUL_DIR,"link.json")
STPAUL_NODE_FILE = os.path.join(STPAUL_DIR,"node.geojson")

## Read in Roadway and Transit Networks

(or open them from a pkl file instead)

In [6]:
road_net = RoadwayNetwork.read(
    link_filename= STPAUL_LINK_FILE, 
    node_filename=STPAUL_NODE_FILE, 
    shape_filename=STPAUL_SHAPE_FILE, 
    fast=True,
    shape_foreign_key="shape_id",
)

2020-07-31 15:03:27, INFO: Reading from following files:
-/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/examples/stpaul/link.json
-/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/examples/stpaul/node.geojson
-/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/examples/stpaul/shape.geojson.
2020-07-31 15:03:27, INFO: Reading from following files:
-/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/examples/stpaul/link.json
-/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/examples/stpaul/node.geojson
-/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/examples/stpaul/shape.geojson.
2020-07-31 15:03:34, INFO: Read 66253 links from /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/examples/stpaul/link.json
2020-07-31 15:03:34, INFO: Read 66253 links from /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/

In [7]:
transit_net = TransitNetwork.read(STPAUL_DIR)

2020-07-31 15:04:08, INFO: Read in transit feed from: /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/examples/stpaul
2020-07-31 15:04:08, INFO: Read in transit feed from: /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/examples/stpaul
2020-07-31 15:04:08, DEBUG: ...agency.txt:
  agency_id       agency_name                   agency_url  agency_timezone  \
0         0     Metro Transit  http://www.metrotransit.org  America/Chicago   
1         2     Metro Transit  http://www.metrotransit.org  America/Chicago   
2         3  Minnesota Valley          http://www.mvta.com  America/Chicago   

  agency_lang  
0          EN  
1          EN  
2          EN  
2020-07-31 15:04:08, DEBUG: ...routes.txt:
  route_id agency_id route_short_name  \
0    3-111         0                3   
1   16-111         2               16   
2   21-111         0               21   
3   53-111         0               53   
4   62-111         0               62   
5

2020-07-31 15:04:08, DEBUG: ...frequencies.txt:
                             trip_id  headway_secs  start_time  end_time
0  14940701-JUN19-MVS-BUS-Weekday-01          3600     21600.0   32400.0
1  14941148-JUN19-MVS-BUS-Weekday-01           830     21600.0   32400.0
2  14941151-JUN19-MVS-BUS-Weekday-01           540     21600.0   32400.0
3  14941153-JUN19-MVS-BUS-Weekday-01           696     32400.0   54000.0
4  14941163-JUN19-MVS-BUS-Weekday-01           830     32400.0   54000.0
5  14942968-JUN19-MVS-BUS-Weekday-01          5400     21600.0   32400.0
6  14943414-JUN19-MVS-BUS-Weekday-01          2160     21600.0   32400.0
7  14943415-JUN19-MVS-BUS-Weekday-01          5400     21600.0   32400.0
8  14944012-JUN19-MVS-BUS-Weekday-01           553     32400.0   54000.0
9  14944018-JUN19-MVS-BUS-Weekday-01           720     21600.0   32400.0
2020-07-31 15:04:08, DEBUG: ...shapes.txt:
  shape_id  shape_pt_lat  shape_pt_lon  shape_pt_sequence shape_osm_node_id  \
0  4520004     44.923257   

## Convert Networks to Mapbox Tiles

1. networks exported as geojson
2. geojson run through the mapbox utility tippecanoe to create more efficient mapbox tiles
3. mapbox tiles are served using mapbox utility mbview 

In [15]:
net_to_mapbox(roadway=road_net,transit=transit_net)

/Users/elizabeth/anaconda3/envs/wrangler/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


2020-07-31 15:17:10, INFO: Running tippecanoe with following options: -zg -o network.mbtiles --force roadway_shapes.geojson transit_shapes.geojson
2020-07-31 15:17:10, INFO: Running tippecanoe with following options: -zg -o network.mbtiles --force roadway_shapes.geojson transit_shapes.geojson
2020-07-31 15:17:47, INFO: Running mbview with following options: -zg -o network.mbtiles --force roadway_shapes.geojson transit_shapes.geojson
2020-07-31 15:17:47, INFO: Running mbview with following options: -zg -o network.mbtiles --force roadway_shapes.geojson transit_shapes.geojson


In [17]:
from IPython.display import IFrame
IFrame('http://localhost:9000', width=900, height=500)